In [1]:
# Import necessary libraries
import json
import os
import re
import sys
from typing import Dict, Any, List

from PyPDF2 import PdfReader
from opensearchpy import OpenSearch, helpers
from sentence_transformers import SentenceTransformer

d:\local_rag\venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
# Set up Python path to access project modules
sys.path.insert(0, "..")

%load_ext autoreload
%autoreload 2

In [ ]:
# Defining constants

# Opensearch connection settings
OPENSEARCH_HOST = "localhost"       # OpenSearch host
OPENSEARCH_PORT = 9200              # OpenSearch port
OPENSEARCH_INDEX = "documents"      # Index name for document storage

# Embeddings settings
EMBEDDING_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"     # Model for generating embedding"
EMBEDDING_DIMENSION = 384                                           # Embedding dimension for the model
ASSYMETRIC_EMBEDDING = False                                        # Whether to use asymmetric embeddings

# Chunking settings
TEXT_CHUNK_SIZE = 500           # Number of tokens per chunk
TEXT_CHUNK_OVERLAP = 100        # Overlap between chunks

print("Constants defined")

Constants defined


In [4]:
# Utility functions for text preprocessing

def clean_text(text: str) -> str:
    """
    Cleans OCR-extracted text by removing unnecessary newlines, hyphens, and correcting common OCR errors.

    Args:
        text (str): The text to clean.

    Returns:
        str: The cleaned text.
    """
    # Remove hyphens at line breaks (e.g., 'exam-\nple' -> 'example')
    text = re.sub(r"(\w+)-\n(\w+)", r"\1\2", text)

    # Replace newlines within sentences with spaces
    text = re.sub(r"(?<!\n)\n(?!\n)", " ", text)

    # Replace multiple newlines with a single newline
    text = re.sub(r"\n+", "\n", text)

    # Remove excessive whitespace
    text = re.sub(r"[ \t]+", " ", text)

    return text.strip()


def chunk_text(text: str, chunk_size: int, overlap: int = 100) -> List[str]:
    """
    Splits text into chunks with a specified overlap.

    Args:
        text (str): The text to split.
        chunk_size (int): The number of tokens in each chunk.
        overlap (int): The number of tokens to overlap between chunks.

    Returns:
        List[str]: A list of text chunks.
    """

    # Clean the text before chunking
    text = clean_text(text)

    # Tokeinze the texts into words
    tokens = text.split(" ")

    chunks = []
    start = 0
    while start < len(tokens):
        end = start + chunk_size
        chunk_tokens = tokens[start:end]
        chunk_text = " ".join(chunk_tokens)
        chunks.append(chunk_text)
        start = end - overlap  # Move back by 'overlap' tokens

    return chunks

print("Utility functions defined")

Utility functions defined


In [5]:
# Embedding functions

def get_embedding_model():
    """
    Loads and returns the sentence transformer embedding model.
    
    Returns:
        SentenceTransformer: The loaded embedding model.
    """
    print(f"Loading embedding model: {EMBEDDING_MODEL_NAME}")
    model = SentenceTransformer(EMBEDDING_MODEL_NAME)
    return model



def generate_embeddings(texts: List[str]):
    """
    Generates embeddings for a list of text chunks.
    
    Args:
        texts (List[str]): List of text chunks to embed.
        
    Returns:
        List[numpy.ndarray]: List of embedding vectors.
    """
    model = get_embedding_model()
    
    # If using asymmetric embeddings, prefix each text with "passage: "
    if ASSYMETRIC_EMBEDDING:
        texts = [f"passage: {text}" for text in texts]
        
    # Generate embeddings
    embeddings = model.encode(texts)
    return embeddings

print("Embedding functions defined")


Embedding functions defined


# 1. Connect to OpenSearch and Create Index